In [9]:
from google import genai
from google.genai import types
import pandas as pd
import re
import json
import json5
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

In [10]:
ems = """
        admiration
        amusement
        anger
        annoyance
        approval
        caring
        confusion
        curiosity
        desire
        disappointment
        disapproval
        disgust
        embarrassment
        excitement
        fear
        gratitude
        grief
        joy
        love
        nervousness
        optimism
        pride
        realization
        relief
        remorse
        sadness
        surprise
        neutral
    """

In [11]:
def emotions_to_categorical(df):
    res = []

    for i in df['emotions']:
        tmp = [0 for _ in range(28)]
        for j in i:
            tmp[j] = 1
        res.append(tmp)
    tmp_df = pd.DataFrame(res, columns=ems.split())
    
    return tmp_df

In [12]:
def data_init(path = "../data/dev.tsv"):
    df = pd.read_csv(path, sep="\t", encoding = "utf-8", header=None)
    df.columns = ['text', 'emotions', 'id']
    df['emotions'] = list(map(lambda s : list(map(int, s.split(','))), df['emotions']))
    df = pd.concat([df, emotions_to_categorical(df)], axis=1)
    df = df.drop(columns=['emotions', 'id'])
    df['text'] = list(map(lambda s : s.replace('\\', '\\\\').replace('"', '\\"'), list(df['text']))) 
    res = df.sample(800).reset_index(drop=True)
    return res

In [13]:
def generate_batch(system, file_name):
    df = data_init()
    file = open(file_name, 'w')
    for i in range(len(df)):
        base_query = f'{{"custom_id": "query{i}", "method": "POST", "url": "/v1/responses", "body": {{"model": "gpt-4o-mini", "input": [{{"role": "developer", "content": "{system}"}}, {{"role": "user", "content": "{df["text"][i]}"}}], "max_output_tokens": 1000, "temperature": 0, "top_logprobs": 5}}}}\n'
        file.write(base_query)
    file.close()
    return df

In [14]:
def status(original, predict):
    check = []
    for i in range(len(original)):
        cnt = 0
        o = 0
        p = 0
        for j in range(len(original[i])):
            if original[i][j] == 1 and predict[i][j] == 1:
                cnt += 1
            if original[i][j] == 1:
                o += 1
            if predict[i][j] == 1:
                p += 1
        check.append([o, p, cnt])
    return check

In [15]:
def evaluation(original_df, emotion_res):
    emotions_list = ems.split()
    df = original_df
    predicted_df = pd.DataFrame(data = [[0 for _ in range(28)] for _ in range(len(df))], columns=emotions_list)
    for i in range(len(emotion_res)):
        for j in emotion_res[i]:
            if j in emotions_list:
                predicted_df.loc[i, j] = 1
    predicted = predicted_df.to_numpy()
    original = df.iloc[:,1:].to_numpy()

    print(original)
    print(predicted)

    c = status(original, predicted)
    
    
    accuracy = accuracy_score(original, predicted)
    
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        original, predicted, average='micro'
    )
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        original, predicted, average='macro'
    )
    
    precision_per_label, recall_per_label, f1_per_label, _ = precision_recall_fscore_support(
        original, predicted, average=None
    )

    precision_macro_std = np.std(precision_per_label)
    recall_macro_std = np.std(recall_per_label)
    f1_macro_std = np.std(f1_per_label)

    print("--- 모델 평가 결과 ---")
    print(f"전체 샘플에 대한 정확도 (Exact Match Accuracy): {accuracy:.4f}")
    print("\n--- Micro 평균 지표 ---")
    print(f"Precision (Micro): {precision_micro:.4f}")
    print(f"Recall (Micro): {recall_micro:.4f}")
    print(f"F1-Score (Micro): {f1_micro:.4f}")
    print("\n--- Macro 평균 지표 ---")
    print(f"Precision (Macro): {precision_macro:.4f}")
    print(f"Recall (Macro): {recall_macro:.4f}")
    print(f"F1-Score (Macro): {f1_macro:.4f}")
    
    print("\n--- 라벨별 지표 ---")
    for i in range(len(emotions_list)):
        print(f"{emotions_list[i]} - Precision: {precision_per_label[i]:.4f}, Recall: {recall_per_label[i]:.4f}, F1-Score: {f1_per_label[i]:.4f}")
    
    print(f"\nPrecision (Macro) 표준편차: {precision_macro_std:.4f}")
    print(f"Recall (Macro) 표준편차: {recall_macro_std:.4f}")
    print(f"F1-Score (Macro) 표준편차: {f1_macro_std:.4f}")

    return accuracy, f1_micro, f1_macro, c

In [17]:
persona = "(PERSONA ROLE) You are an expert emotion analyst with deep expertise in linguistics and psychology. Your task is to classify the given text into one or more of the following 28 fine-grained GoEmotions categories: [admiration, amusement, anger, annoyance, approval, caring, confusion, curiosity, desire, disappointment, disapproval, disgust, embarrassment, excitement, fear, gratitude, grief, joy, love, nervousness, optimism, pride, realization, relief, remorse, sadness, surprise, neutral]. Your primary function is to analyze English text and classify its emotional content with precision and objectivity. (PERSONA ROLE END)"

In [18]:
guidelines = "(GUIDELINES) 1. Think step by step carefully, and ONLY output final results in strict JSON. 2. Assign **at least 1** and **at most 5** emotion labels per input sentence. 3. In most cases, a sentence should have **1 or 2 dominant emotions** only. 4. Use multiple labels only when the text clearly expresses overlapping emotions. 5. Avoid over-labeling; choose the most relevant categories. 6. Neutral should be chosen only when: - The text contains no clear emotional cues, OR - The content is purely factual or descriptive without emotional tone, OR - Emotional signals are too weak or ambiguous to assign a specific emotion. 7. Do NOT assign neutral if: - A clear positive or negative emotion is expressed, even mildly. - The sentence mixes factual and emotional elements: in that case, pick the dominant emotion(s). (GUIDELINES END)"

In [19]:
output_structure = "(OUTPUT STRUCTURE) 1. Strict JSON Output: Your final output must be a single JSON object. Do not include any text outside of this JSON object. 2. JSON Structure: The JSON object must contain a key named analysis which is an array of objects. Each object in the array represents a single identified emotion and must contain the following two keys: -emotion: The specific emotion label from the list. -reason: A brief, objective explanation of why you chose this emotion, citing specific words or phrases from the text. (OUTPUT STRUCTURE END)"

In [20]:
few_shot_example = """
(FEW_SHOT_EXAMPLE)
[
    {
        "example_id": 1,
        "input": "They really aren't people. They were slavers. Civilized people should have only one answer to that sort of behaviour: violence and death.",
        "output": {
            "analysis": [
                {
                    "emotion": "grief",
                    "reason": "The text expresses a deep sense of sorrow and moral injury over the historical actions of 'slavers', framing the extreme response as a consequence of that profound grief."
                }
            ]
        }
    },
    {
        "example_id": 2,
        "input": "random fact: my pokerstars avatar has been a picture of that very hit since that night of 2006 LOL you could say that this hit was... pretty mint",
        "output": {
            "analysis": [
                {
                    "emotion": "pride",
                    "reason": "The user shows pride by highlighting that their avatar has commemorated a 'pretty mint' achievement for a very long time (since 2006)."
                }
            ]
        }
    },
    {
        "example_id": 3,
        "input": "You're dehydrated and exausted, drink lots of water and maybe get some sleep. You'll feel better eventually.",
        "output": {
            "analysis": [
                {
                    "emotion": "relief",
                    "reason": "The statement 'You'll feel better eventually' provides reassurance and promises relief from the current state of being dehydrated and exhausted."
                }
            ]
        }
    },
    {
        "example_id": 4,
        "input": "Quitting porn only helped with my performance in bed, other than that I'm still anxious as hell in social spots",
        "output": {
            "analysis": [
                {
                    "emotion": "nervousness",
                    "reason": "The text explicitly mentions the feeling of being 'anxious as hell', which is a direct indicator of nervousness."
                }
            ]
        }
    },
    {
        "example_id": 5,
        "input": "In that conversation it was the girl being boring and closed... sometimes it's not your fault and people feel they're the ones being awkward",
        "output": {
            "analysis": [
                {
                    "emotion": "embarrassment",
                    "reason": "The text addresses the feeling of being 'awkward', which is closely related to embarrassment, and attempts to alleviate that feeling."
                }
            ]
        }
    },
    {
        "example_id": 6,
        "input": "This was probably one of the best videos I've ever seen on Reddit. What an amazing story. Job well done. RIP Chief.",
        "output": {
            "analysis": [
                {
                    "emotion": "admiration",
                    "reason": "High praise like 'one of the best videos', 'amazing story', and 'Job well done' clearly expresses admiration."
                },
                {
                    "emotion": "grief",
                    "reason": "The phrase 'RIP Chief' is a direct expression of grief and mourning for someone who has passed away."
                }
            ]
        }
    },
    {
        "example_id": 7,
        "input": "Your dog provided you undeniable evidence of type of person that you are, and you should be proud of that.",
        "output": {
            "analysis": [
                {
                    "emotion": "pride",
                    "reason": "The text directly encourages the listener to feel proud with the explicit instruction 'you should be proud of that'."
                },
                {
                    "emotion": "neutral",
                    "reason": "The first clause, 'Your dog provided you undeniable evidence...', is presented as a factual observation without inherent emotional tone."
                }
            ]
        }
    },
    {
        "example_id": 8,
        "input": "We just got a ninja coffee machine. Don't even need filters! It has little waste besides the coffee grounds. I love it.",
        "output": {
            "analysis": [
                {
                    "emotion": "love",
                    "reason": "The user explicitly states their positive feeling for the new machine with the phrase 'I love it'."
                },
                {
                    "emotion": "relief",
                    "reason": "The statement 'Don't even need filters!' indicates that a previous hassle has been removed, suggesting a feeling of relief."
                }
            ]
        }
    },
    {
        "example_id": 9,
        "input": "So glad you are safe. It looks like icy conditions, too, so that is great that you were able to avoid this.",
        "output": {
            "analysis": [
                {
                    "emotion": "caring",
                    "reason": "The text expresses concern for the person's safety, especially in the context of 'icy conditions'."
                },
                {
                    "emotion": "gratitude",
                    "reason": "The phrase 'So glad' indicates a feeling of thankfulness that the person is safe."
                },
                {
                    "emotion": "relief",
                    "reason": "The core message 'So glad you are safe' communicates relief that a potential danger was averted."
                }
            ]
        }
    },
    {
        "example_id": 10,
        "input": "Where the mask inside, too? Are air purifers not good enough for inside? Sorry for the dumb question... Just moved here this week.",
        "output": {
            "analysis": [
                {
                    "emotion": "curiosity",
                    "reason": "The user is asking direct questions ('Where...?', 'Are... not good enough?') to gather information."
                },
                {
                    "emotion": "remorse",
                    "reason": "The apology 'Sorry for the dumb question' shows remorse for potentially asking something inappropriate or obvious."
                },
                {
                    "emotion": "sadness",
                    "reason": "The context 'Just moved here this week' can imply a feeling of loneliness or being unsettled, which is a form of sadness."
                }
            ]
        }
    }
]
(FEW_SHOT_EXAMPLE END)""".replace('  ', "").replace('   ', "").replace('    ', "").replace('     ', "").replace('      ', "").replace('\n'," ").replace('\t', " ").replace('"', "'").replace('{ ', '{').replace('[ ', '[').replace(' }', '}').replace(' ]', ']')

In [21]:
chain_of_thought = """
[CHAIN OF THOUGHT]
Read the entire Text thoroughly.
Follow the Thought Process internally and draw out the final emotion.
Text: [Text example] 
Thought Process: 
Text Analysis: [Describe the mood or the main content of this text. For example, "The sentence is a question expressing uncertainty and confusion."] 
Identify Clues: [Find words or phrases in the text that clearly express emotion. For example, "The phrase 'I don’t know anybody...' directly indicates a lack of knowledge or understanding, leading to confusion."] 
Final Judgment: [Synthesize the clues to explain why you concluded this emotion. For example, "The questioning tone and lack of information point directly to the emotion of 'confusion'."] 
Emotion: [Emotion label]
example 1
Text: Your response is truly one of the most amazing ones I’ve seen.
Thought Process:
Text Analysis: The sentence is an extremely positive statement directed at another person's work or contribution ("Your response"). The tone is one of high praise and commendation.
Identify Clues: The phrase "truly one of the most amazing ones" is a strong form of praise. Words like "truly" and "amazing" are explicit indicators of a strong, positive feeling and a high regard for the quality of the response.
Final Judgment: The speaker is expressing profound approval and respect for the quality of the response. This is a clear expression of 'admiration'.
Emotion: admiration
example 2
Text: Haha this dude’s from the area I’m from. He’s a local treasure 
Thought Process:
Text Analysis: The sentence uses informal, familiar language ("Haha," "dude's," "local treasure") to express positive sentiment about a person.
Identify Clues: "Haha" is a direct indicator of laughter and amusement. "Local treasure" is a term of endearment, suggesting the person is valued and brings joy.
Final Judgment: The combination of laughter and positive, informal praise indicates a feeling of lightheartedness and pleasure. 
Emotion: amusement
example 3
Text: WHY THE FUCK IS BAYLESS ISOING 
Thought Process:
Text Analysis: This is an aggressive and frustrated question. The use of a profanity and all-caps text shows strong negative emotion.
Identify Clues: The use of "FUCK" and the question in all caps highlights intense frustration and displeasure with a person's actions. "ISOING" implies a specific action that the speaker disapproves of.
Final Judgment: The raw, aggressive language clearly demonstrates intense displeasure and exasperation.
Emotion: anger 
example 4
Text: Shit, I guess I accidentally bought a Pay-Per-View boxing match 
Thought Process:
Text Analysis: The speaker realizes they made a mistake ("accidentally bought") that has negative consequences (a purchase they didn't intend to make).
Identify Clues: The expletive "Shit" at the beginning is a common expression of frustration or mild exasperation. The word "accidentally" confirms the lack of intent.
Final Judgment: The text conveys a feeling of being bothered and irritated by a careless mistake.
Emotion: annoyance
example 5
Text: You can always kneel. 
Thought Process:
Text Analysis: This is a brief, declarative statement offering a suggestion or alternative. It implies a sense of calm and acceptance.
Identify Clues: The phrase "You can always" suggests a simple, uncontroversial solution. It implies that the action is both possible and acceptable.
Final Judgment: The text provides an alternative without judgment, suggesting that the speaker supports or agrees with the action. 
Emotion: approval
example 6
Text: Glad you feel better! My offer still stands though, if you need someone, I’m here Thought Process:
Text Analysis: The sentence begins with a positive wish for the other person's well-being and follows up with an offer of continued support.
Identify Clues: "Glad you feel better!" is an expression of happiness for someone else's recovery. "My offer still stands though, if you need someone, I’m here" is a direct statement of support and willingness to help.
Final Judgment: The combination of happiness for the other person and the offer of help indicates a genuine and supportive attitude. 
Emotion: caring
example 7
Text: Demographics? I don’t know anybody under 35 who has cable tv. 
Thought Process:
Text Analysis: The speaker is responding to a question about demographics by stating their personal lack of knowledge and perspective.
Identify Clues: The question "Demographics?" and the statement "I don't know anybody under 35..." show the speaker's personal experience contradicts the premise of the question, leading to a state of being baffled.
Final Judgment: The speaker is expressing uncertainty and a lack of understanding based on their own observations. 
Emotion: confusion
example 8
Text: Who is this Wild team? Where have they been? 
Thought Process:
Text Analysis: This is a series of questions about a sports team. The questions aren't rhetorical; they're seeking information.
Identify Clues: "Who is this...?" and "Where have they been?" are direct questions aimed at gathering information and satisfying a desire to know more about the subject.
Final Judgment: The speaker's questions reveal an interest in learning more about the team's background and performance. 
Emotion: curiosity
example 9
Text: I want my nuggs mom! 
Thought Process:
Text Analysis: The speaker is making a direct and demanding request for something they want.
Identify Clues: The phrase "I want my nuggs" is a clear statement of desire for a specific item.
The word "mom!" can be used to emphasize the demand.
Final Judgment: The text is an explicit expression of wanting something intensely. 
Emotion: desire
example 10
Text: He was off by 5 minutes, not impressed. 
Thought Process:
Text Analysis: The sentence states a minor failure ("off by 5 minutes") and explicitly links it to a negative feeling.
Identify Clues: The phrase "not impressed" is a direct statement of negative feeling due to a perceived shortcoming.
Final Judgment: The speaker is expressing a feeling of dissatisfaction and letdown because of a minor failure. 
Emotion: disappointment
example 11
Text: This isn't really wholesome 
Thought Process:
Text Analysis: The speaker is providing a negative assessment of a situation.
Identify Clues: "isn't really wholesome" is a form of negative judgment. It suggests that the speaker finds the content to be inappropriate or unsavory.
Final Judgment: The text is a criticism of something that fails to meet a standard of "wholesomeness," indicating a lack of approval. 
Emotion: disapproval
example 12
Text: ur a disgusting piece of filth. 
Thought Process:
Text Analysis: This is a direct, insulting statement directed at another person. The language is extremely negative and offensive.
Identify Clues: The words "disgusting" and "filth" are strong expressions of revulsion and contempt.
Final Judgment: The text uses powerful, negative descriptors to show revulsion and strong moral contempt for the person being addressed. 
Emotion: disgust
example 13
Text: i feel dumb <3 
Thought Process:
Text Analysis: The speaker is expressing a negative self-perception, yet follows it with a heart emoji. This contrast is key.
Identify Clues: "i feel dumb" is a direct statement of feeling foolish or unintelligent. The heart emoji softens the tone, suggesting a self-deprecating or relatable statement.
Final Judgment: The speaker is admitting to a foolish mistake or action and feeling silly about it. 
Emotion: embarrassment
example 14
Text: Yeah let's wrap up AS4 I'm suddenly so much more excited for S11 
Thought Process:
Text Analysis: The speaker expresses a desire to move on from a current event to a new one, showing a surge of enthusiasm.
Identify Clues: "suddenly so much more excited" is a direct, explicit statement of heightened anticipation and positive emotion.
Final Judgment: The language clearly indicates a high level of enthusiasm and anticipation for a future event. 
Emotion: excitement
example 15
Text: To make her feel threatened 
Thought Process:
Text Analysis: This is a brief, purpose-driven statement. It describes an action intended to elicit a specific negative emotional response from someone else.
Identify Clues: The verb "threatened" is a direct synonym for the emotion of fear. The action's intent is to cause this feeling.
Final Judgment: The statement explicitly mentions the goal of causing a feeling of being threatened. 
Emotion: fear
example 16
Text: Thank you friend 
Thought Process:
Text Analysis: The speaker is expressing thanks to a person they consider a friend.
Identify Clues: The phrase "Thank you" is a direct and universal expression of gratitude. The addition of "friend" makes the gratitude feel more personal and warm.
Final Judgment: The text is a clear and direct expression of thankfulness. 
Emotion: gratitude
example 17
Text: I read on a different post that he died shortly after of internal injuries. 
Thought Process:
Text Analysis: The sentence conveys news of someone's death and a tragic cause of death.
Identify Clues: The subject of the sentence is "he died," which is a direct reference to death, a primary cause of grief. "Internal injuries" adds a painful detail.
Final Judgment: The text reports a tragic loss, a topic inherently associated with sorrow and profound sadness. 
Emotion: grief
example 18
Text: Happy to be able to help. 
Thought Process:
Text Analysis: The speaker is expressing positive feelings about an action they performed.
Identify Clues: "Happy to be able to help" is a direct and simple statement of joy from being helpful. The word "happy" is a clear emotional marker.
Final Judgment: The speaker is expressing happiness and satisfaction from assisting someone. 
Emotion: joy
example 19
Text: I adore this comment section <3 
Thought Process:
Text Analysis: The speaker is expressing a very strong positive feeling towards a group of comments.
Identify Clues: The word "adore" is a powerful term for deep affection or love. The heart emoji "<3" reinforces this feeling.
Final Judgment: The text is an explicit expression of deep affection and fondness. 
Emotion: love
example 20
Text: I am getting so nervous watching this 
Thought Process:
Text Analysis: The speaker is describing a physical or emotional state related to an ongoing event.
Identify Clues: The phrase "getting so nervous" is a direct, explicit statement of a feeling of anxiety or fear.
Final Judgment: The speaker is clearly articulating a state of anxiety or apprehension while observing something. 
Emotion: nervousness
example 21
Text: I hope she lives to be 200, just to piss them off. 
Thought Process:
Text Analysis: The speaker is expressing a positive hope for someone's future, but with a humorous, spiteful motivation.
Identify Clues: The phrase "I hope she lives to be 200" is a positive, future-oriented statement. While the motive "to piss them off" adds a twist, the primary emotion is about holding a positive outlook for the future.
Final Judgment: The text is a statement of hope for a positive outcome in the future, even if the motive is negative. 
Emotion: optimism
example 22
Text: I'm so proud of you! 
Thought Process:
Text Analysis: The speaker is directly conveying a strong positive emotion about another person's achievement.
Identify Clues: "I'm so proud of you!" is a direct, explicit statement of pride.
Final Judgment: The speaker is clearly expressing a feeling of satisfaction in another person's accomplishment.
Emotion: pride
example 23
Text: I take that back after the last two games. 
Thought Process:
Text Analysis: The speaker is retracting a previous statement due to new information orevidence.
Identify Clues: The phrase "I take that back" indicates that the speaker's prior opinion has been changed by recent events ("after the last two games"). This is a sudden recognition of a new truth.
Final Judgment: The speaker has a sudden new understanding of a situation, which is a classic example of realization. 
Emotion: realization
example 24
Text: I’m glad I wasn’t the only one. I thought wow they’re really gonna get it for this one Thought Process:
Text Analysis: The speaker expresses relief that they were not alone in a feeling or belief. The phrase "wow they're really gonna get it for this one" indicates that they were expecting a negative outcome.
Identify Clues: "I'm glad I wasn't the only one" is a direct statement of relief. The underlying
fear of being alone in a risky opinion has been alleviated.
Final Judgment: The speaker is feeling a sense of comfort and ease now that they know others share their opinion, which is a textbook example of relief.
Emotion: relief
example 25
Text: Apologies, I take it all back as I’ve just seen his latest effort 
Thought Process:
Text Analysis: The speaker is apologizing and retracting a previous statement after a new observation.
Identify Clues: The word "Apologies" is a direct apology. "I take it all back" is a statement of regret, acknowledging a mistake in judgment.
Final Judgment: The speaker is expressing regret and remorse for their past words. Emotion: remorse
example 26
Text: my brain hurts... 
Thought Process:
Text Analysis: This is a simple statement of physical discomfort, likely caused by a mentally taxing or emotionally draining situation.
Identify Clues: "my brain hurts..." can be a literal statement of physical pain, but in this context, it is a common idiom for feeling overwhelmed or upset. It signifies distress and mental exhaustion.
Final Judgment: The speaker is expressing a feeling of mental or emotional pain, which is an indicator of sadness. 
Emotion: sadness
example 27
Text: This...has 9k upvotes. Wow. 
Thought Process:
Text Analysis: The speaker is reacting to an unexpected and highly unusual number of upvotes.
Identify Clues: The ellipsis "..." and the exclamation "Wow." are direct exclamations of shock or astonishment. The number "9k" is presented as a surprisingly large figure.
Final Judgment: The speaker is expressing a feeling of being taken aback by an unexpected fact. 
Emotion: surprise
example 28
Text: My favourite food is anything I didn't have to cook myself. 
Thought Process:
Text Analysis: This is a statement of personal preference about food. It does not contain any strong emotional cues.
Identify Clues: There are no words or phrases that indicate a specific positive or negative emotion. The statement is factual and declarative.
Final Judgment: The text is a simple statement of fact or opinion without any emotional undertone. 
Emotion: neutral
[CHAIN OF THOUGHT END]
""".replace('\n', ' ').replace('"', "'")

In [22]:
system = f"We are trying to classify 28 emotions of goemotion, please read the prompt below carefully and proceed with the classification without errors. {persona}, {guidelines}, {chain_of_thought}, {output_structure}, {few_shot_example}"

In [ ]:
file_name = "input_0_0_1.jsonl"
original_df = generate_batch(system, file_name)

In [7]:
key_file = open('./gemini_key.txt', 'r')
api_key = key_file.readline()
key_file.close()

In [8]:
client = genai.Client(api_key=api_key)

In [ ]:
uploaded_file = client.files.upload(
    file = './' + file_name
    config = types.UploadFileConfig(display_name=file_name, mime_type='jsonl')
)
print(f"Uploaded file: {uploaded_file.name}")

In [ ]:
file_batch_job = client.batches.create(
    model="gemini-2.5-flash",
    src=uploaded_file.name,
    config={
        'display_name': file_name,
    },
)

print(f"Created batch job: {file_batch_job.name}")